# Brief **Introduction au webscraping**

**Auteur**: César Gattano

## Rappel des consignes du projet

Pour l'étude de la certification, plusieurs parcours d'apprentissage sont proposé par Microsoft Learn, chaque parcours d'apprentissage est constitué de modules, chaque module est constitué de plusieurs page de cours, dans plusieurs pages vous retrouverez des paragraphes, des tableaux.
Il est demandé de récupérer chaque page de cours de chaque module pour chacun des parcours d'apprentissage de la certification.
Stockez l'ensemble dans une base de données MongoDB (en local ou sur Atlas). Proposez un schéma de stockage qui vous semblera pertinent.

Restituez l'ensemble de vos travaux sur un notebook jupyter (extraction, aggrégation des données et mise en base sur mongoDB).
Utilisez requests pour les requêtes HTTP, beautifulsoup pour le parsing HTML et le client python pymongo pour stocker en base MongoDB.

Pour orienter vos recherches :
Définissez :
- Attaque DDOS
- Arborescence DOM
- Elément enfant / parent / frère
- Balise HTML inline
- Nom d'une balise HTML
- Attribut de balise et valeur d'attribut HTML
- Différences entre id et class sur une balise HTML
- Format BJSON

De plus, je souhaite que vous réfléchissiez sur les questions suivantes :
- Qu'est-ce que le webscraping ?
- Est-ce que le webscraping est légal ?
- Quelles sont les bonnes pratiques ?
- Qu'est-ce qui est proscrit ?

In [1]:
# Importation selenium requise pour l'éxecution du notebook
from selenium import webdriver

# Importation beautiful soup requise pour l'éxecution du notebook
from bs4 import BeautifulSoup, NavigableString, PageElement
from bs4._typing import _AtMostOneElement

# Importation pymongo requise pour l'éxecution du notebook
from pymongo import MongoClient

# Autres importations requises
import time
import re

## **selenium**: requête serveur pour la récupération du contenu de la page

On définit en premier lieu l'url et les options pour le navigateur Chrome de **selenium**

In [2]:
url = "https://learn.microsoft.com/fr-fr/training/courses/dp-203t00"

options = webdriver.ChromeOptions()
options.add_argument('headless')

On crée le navigateur chrome et on lance la requête pour accéder à la page initiale

In [3]:
browser = webdriver.Chrome(options=options)
browser.get(url)
# Attente de 2sec pour être sûr d'avoir chargé toute la page
time.sleep(2)

## **beautiful soup**: analyse du contenu HTML de la page pour extraction d'information

On transforme le contenu de la page pour pouvoir opérer le webscraping avec **beautiful soup**.

In [4]:
soup = BeautifulSoup(browser.page_source, "html.parser")

### Webscrapping sur la page initiale

Sur cette première page la seule tâche et de récupérer les url pour chaque parcours d'apprentissage et le titre de ces parcours. Pour cela, on filtre les balises de manière a isoler les seules balises qui contiennent ces informations.

In [5]:
for tag in soup.find_all("a", class_="card-title", attrs={"data-bi-name": "study-guide"}, href=re.compile("^/fr-fr/training")):
    url = "https://learn.microsoft.com" + tag.get_attribute_list("href")[0]
    title = tag.get_text(strip=True)
    print(url, "\n", title)

https://learn.microsoft.com/fr-fr/training/paths/get-started-data-engineering/ 
 Commencez avec l’ingénierie des données sur Azure
https://learn.microsoft.com/fr-fr/training/paths/build-data-analytics-solutions-using-azure-synapse-serverless-sql-pools/ 
 Créer des solutions d’analytique données avec des pools SQL serverless Azure Synapse
https://learn.microsoft.com/fr-fr/training/paths/perform-data-engineering-with-azure-synapse-apache-spark-pools/ 
 Appliquer l’Engineering données avec des pools Azure Synapse Apache Spark
https://learn.microsoft.com/fr-fr/training/paths/transfer-transform-data-azure-synapse-analytics-pipelines/ 
 Transférer et transformer des données avec des pipelines Azure Synapse Analytics
https://learn.microsoft.com/fr-fr/training/paths/implement-data-analytics-azure-synapse-analytics/ 
 Implémenter une solution d’analytique données avec Azure Synapse Analytics
https://learn.microsoft.com/fr-fr/training/paths/work-with-data-warehouses-using-azure-synapse-analytics

Pour commencer, on va se pencher sur le premier url extrait et lancer une nouvelle requête au navigateur

In [6]:
tag = soup.find("a", class_="card-title", attrs={"data-bi-name": "study-guide"}, href=re.compile("^/fr-fr/training"))
url = "https://learn.microsoft.com" + tag.get_attribute_list("href")[0]
print(f"Changement de page: {url}")
browser.get(url)
time.sleep(2)
soup_p1 = BeautifulSoup(browser.page_source, "html.parser")

Changement de page: https://learn.microsoft.com/fr-fr/training/paths/get-started-data-engineering/


### Webscrapping sur la page du premier parcours d'apprentissage

L'information que l'on recherche est concentré dans une balise de la class *modular-content-container*. On inspecte les textes qu'elle contient.

In [7]:
contents = soup_p1.find_all("div", class_="modular-content-container")

for string in contents[0].stripped_strings:
    print(string)


2400 XP
Commencez avec l’ingénierie des données sur Azure
2 h 18 min
Parcours d’apprentissage
3 Modules
Intermédiaire
Ingénieur Data
Azure Data Lake Storage
Azure Synapse Analytics
Dans la plupart des organisations, un ingénieur Données est le rôle principal responsable de l’intégration, de la transformation et du regroupement des données de divers systèmes de données structurés et non structurés dans des structures adaptées à la création de solutions d’analytique. Les Ingénieurs Données Azure veillent également à ce que les pipelines de données et les magasins de données restent très performants, efficaces, organisés et fiables, en fonction d’un ensemble spécifique d’exigences et de contraintes stratégiques.
Prérequis
Avant de commencer ce parcours d’apprentissage, vous devez avoir terminé la
certification Microsoft Azure Data Fundamentals
ou avoir une connaissance et une expérience équivalentes.
Démarrer
Ajouter
Ajouter à des collections
Ajouter au plan
Ajouter aux défis
Code de réus

On récupère les informations pertinentes correspondant à un parcours d'apprentissage

In [8]:
soup_p1_general = contents[0]
list_strings = list(soup_p1_general.stripped_strings)

title = list_strings[1]
print(f"Titre: {title}")

duration = list_strings[2]
print(f"Durée: {duration}")

nb_module = int(list_strings[4].split()[0])
print(f"nombre de modules: {nb_module}")

idx = list_strings.index("Prérequis")
syllabus = list_strings[idx-1]
print(f"Avant-propos: {syllabus}")

level = list_strings[5]
print(f"Niveau: {level}")

keywords = list_strings[6:idx-1]
print(f"Mots-clés: {keywords}")

idx2 = list_strings.index("Démarrer")
prerequisite = " ".join(list_strings[idx+1:idx2])
print(f"Prérequis: {prerequisite}")


Titre: Commencez avec l’ingénierie des données sur Azure
Durée: 2 h 18 min
nombre de modules: 3
Avant-propos: Dans la plupart des organisations, un ingénieur Données est le rôle principal responsable de l’intégration, de la transformation et du regroupement des données de divers systèmes de données structurés et non structurés dans des structures adaptées à la création de solutions d’analytique. Les Ingénieurs Données Azure veillent également à ce que les pipelines de données et les magasins de données restent très performants, efficaces, organisés et fiables, en fonction d’un ensemble spécifique d’exigences et de contraintes stratégiques.
Niveau: Intermédiaire
Mots-clés: ['Ingénieur Data', 'Azure Data Lake Storage', 'Azure Synapse Analytics']
Prérequis: Avant de commencer ce parcours d’apprentissage, vous devez avoir terminé la certification Microsoft Azure Data Fundamentals ou avoir une connaissance et une expérience équivalentes.


Dans la perspective d'une mise en base de données ultérieure, on va aggréger toutes ces informations dans un dictionnaire

In [9]:
new_learning_path = {
    "titre": title,
    "duree": duration,
    "avant_propos": syllabus,
    "niveau": level,
    "mots_cles": keywords,
    "prerequis": prerequisite,
    "module_ids": [],
}
# Les module_ids seront complétées plus tard durant la mise en base de données.

Pour le reste de cette page, on va simplement aller chercher les url des pages de présentation des différents modules composant le parcours d'apprentissage.

In [10]:
soup_p1_modules = contents[1]
link_lists = []
for tag in soup_p1_modules.find_all(href=re.compile("^../../modules")):
    link_lists.append(tag.get_attribute_list("href")[0])
for link in link_lists:
    nb_occurences = link_lists.count(link)
    for _ in range(nb_occurences-1):
        link_lists.remove(link)
link_lists = ["https://learn.microsoft.com/fr-fr/training/" + link[6:] for link in link_lists]
link_lists


['https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/',
 'https://learn.microsoft.com/fr-fr/training/modules/introduction-to-azure-data-lake-storage/',
 'https://learn.microsoft.com/fr-fr/training/modules/introduction-azure-synapse-analytics/']

À nouveau, on va se limiter au premier url pour le premier module afin d'en extraire les informations pertinentes

In [11]:
url = link_lists[0]
print(f"Changement de page vers {url}...")
browser.get(url)
time.sleep(2)
soup_p1_m1 = BeautifulSoup(browser.page_source, "html.parser")

Changement de page vers https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/...


### Webscrapping sur la page du premier module

À nouveau l'ensemble de l'information pertinente est concentrée dans une balise de la classe *modular-content-container*.

In [12]:
content = soup_p1_m1.find("div", class_="modular-content-container")

for string in content.stripped_strings:
    print(string)


700 XP
Présentation de l’engineering données dans Azure
26 min
Module
6 Unités
Commentaires
Intermédiaire
Ingénieur Data
Azure
Microsoft Azure fournit une plateforme complète pour l’engineering données, mais qu’est-ce que l’engineering données ? Suivez ce module pour le découvrir.
Objectifs d’apprentissage
Dans ce module, vous allez apprendre à :
Identifier les tâches courantes d’ingénierie des données
Décrire les concepts courants d’ingénierie des données
Identifier les services Azure pour l’ingénierie des données
Démarrer
Ajouter
Ajouter à des collections
Ajouter au plan
Ajouter aux défis
Prérequis
Avant de commencer ce module, vous devez avoir effectué la
certification Microsoft Azure Data Fundamentals
ou avoir des connaissances et une expérience équivalentes.
Prise en main d’Azure
Choisissez le compte Azure qui vous convient. Payez à l’utilisation ou essayez Azure gratuitement pendant jusqu’à 30 jours.
S’inscrire.
Ce module fait partie de ces parcours d’apprentissage
Commencez avec

On extrait l'information pertinente à partir de la liste de strings.

In [13]:
soup_p1_m1_general = content
list_strings = list(soup_p1_m1_general.stripped_strings)

title = list_strings[1]
print(f"Titre: {title}")

duration = list_strings[2]
print(f"Durée: {duration}")

nb_units = int(list_strings[4].split()[0])
print(f"nombre d'unités: {nb_units}")

idx = list_strings.index("Objectifs d’apprentissage")
syllabus = list_strings[idx-1]
print(f"Avant-propos: {syllabus}")

level = list_strings[6]
print(f"Niveau: {level}")

keywords = list_strings[7:idx-1]
print(f"Mots-clés: {keywords}")

idx2 = list_strings.index("Démarrer")
goals = list_strings[idx+2:idx2]
print(f"Objectifs: {goals}")

idx3 = list_strings.index("Prérequis")
idx4 = list_strings.index("Prise en main d’Azure")
prerequisite = " ".join(list_strings[idx3+1:idx4])
print(f"Prérequis: {prerequisite}")

idx5 = list_strings.index("Ce module fait partie de ces parcours d’apprentissage")
idx6 = list_strings.index("Passer l’évaluation du module")
belongs_to = list_strings[idx5+1:idx6]
print(f"Fait partie des parcours: {belongs_to}")

Titre: Présentation de l’engineering données dans Azure
Durée: 26 min
nombre d'unités: 6
Avant-propos: Microsoft Azure fournit une plateforme complète pour l’engineering données, mais qu’est-ce que l’engineering données ? Suivez ce module pour le découvrir.
Niveau: Intermédiaire
Mots-clés: ['Ingénieur Data', 'Azure']
Objectifs: ['Identifier les tâches courantes d’ingénierie des données', 'Décrire les concepts courants d’ingénierie des données', 'Identifier les services Azure pour l’ingénierie des données']
Prérequis: Avant de commencer ce module, vous devez avoir effectué la certification Microsoft Azure Data Fundamentals ou avoir des connaissances et une expérience équivalentes.
Fait partie des parcours: ['Commencez avec l’ingénierie des données sur Azure']


À nouveau, dans la perspective d'une mise en base de données, nous allons aggréger ces informations dans un dictionnaire

In [14]:
new_module = {
    "titre": title,
    "duree": duration,
    "avant_propos": syllabus,
    "niveau": level,
    "mots_cles": keywords,
    "objectifs": goals,
    "prerequis": prerequisite,
    "parcours_ids": [],
    "notion_ids": [],
}
# Les parcours_ids et notion_ids seront complétés ultérieurement durant la mise en base

Pour le reste de cette page, on va simplement extraire les chemins relatifs des différentes unités de cours pour obtenir leur url

In [15]:
unit_list_tag = soup_p1_m1.find('ul', id="unit-list")
link_lists = []
for tag in unit_list_tag.find_all(href=True):
    link_lists.append(url + tag.get_attribute_list("href")[0])
link_lists

['https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/1-introduction',
 'https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/2-what-data-engineering',
 'https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/4-common-patterns-azure-data-engineering',
 'https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/5-common-tooling-azure-data-engineering',
 'https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/6-knowledge-check',
 'https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/7-summary']

Pour la suite, on va se concentrer sur l'un de ces urls qui redirige vers une unité un tant soit peu complexe afin d'en extraire son contenu

In [16]:
url = link_lists[1]
print(f"Changement de page vers {url}...")
browser.get(url)
time.sleep(2)
soup_p1_m1_u2 = BeautifulSoup(browser.page_source, "html.parser")

Changement de page vers https://learn.microsoft.com/fr-fr/training/modules/introduction-to-data-engineering-azure/2-what-data-engineering...


## Webscrapping sur la page d'unité de cours sélectionnée

Cette fois-ci, on extrait le contenu grâce à la classe *content*.

In [17]:
content = soup_p1_m1_u2.find("div", class_="content")

Au début, on suppose que la structure de la page présente des éléments systématiques quelques soit la page d'unité inspectée: la balise de titre de l'unité de cours et les metadata.

In [18]:
tag = content.find(id='module-unit-title')
tag = tag.next_element
title = str(tag)
print(f"titre: {title}")

# On saute aux metadata de l'unité de cours également systématique
tag = content.find(id="module-unit-metadata")
tag = tag.next_element
metadata = []
while tag:
    if not isinstance(tag, NavigableString):
        if tag.get_attribute_list("id")[0] == "module-unit-total-time":
            duration = str(tag.string)
        else:
            metadata.append(str(tag.string))
    tag = tag.next_sibling
print(f"Durée: {duration}")
print(f"Metadata: {metadata}")


titre: Qu’est-ce que l’ingénierie des données ?
Durée: 5 minutes
Metadata: []


Ensuite on s'attaque au bloc de contenu... mais avant ça, on voit que souvent, des **NavigableString** contenant uniquement un retour chariot '\n' parasite un peu la recherche de contenu. Une fonction pour permettre de les ignorer serait bienvenue.

In [19]:
def not_useless_navigable_string(page_element: PageElement) -> bool:
    """Identify if a PageElement is not a useless NavigableString:
    - '\n'

    Args:
        page_element (PageElement): The page element

    Returns:
        bool: is the page element a useless NavigableString ?
    """
    return not(isinstance(page_element, NavigableString) and page_element == '\n')


La structure du contenu d'une unité de cours pouvant varier d'une unité à l'autre, il est nécessaire de développer un code qui doit pouvoir s'adapter à tous les cas. Le long bloc de code va inspecter l'ensemble des enfants de la balise *content* et va les analyser pour en extraire leur contenu et les aggréger au sein d'un dictionnaire.

In [20]:
# Récupération des balises enfants
tag = content.find(id="module-unit-content")
contents = list(filter(not_useless_navigable_string, tag.children))

# Initialisation du dictionnaire (on ajout les informations systématiques extraites ci-dessus)
unit_content = {}
unit_content["titre"] = title
unit_content["duree"] = duration
if len(metadata) > 0:
    unit_content["metadata"] = metadata
unit_content["texte"] = []
unit_content["parts"] = []

# Initialisation de booléens
a_part_is_opened = False
a_subpart_is_opened = False

# Début de l'inspection
for tag in contents:
    print(tag.name)

    if tag.name == 'p':      
        # Traitement d'un paragraphe:
        paragraph = " ".join(tag.stripped_strings).strip()

        if paragraph:
            # Redirection vers le niveau correspondant du cours
            if a_part_is_opened:
                if a_subpart_is_opened:
                    unit_subpart["texte"].append(paragraph)
                else:
                    unit_part["texte"].append(paragraph)
            else:
                unit_content["texte"].append(paragraph)

    elif tag.name == 'h2':
        # Traitement d'un titre de partie:

        # Stop the current part ?
        if a_part_is_opened:
            # Stop the current subpart ?
            if a_subpart_is_opened:
                unit_part["subparts"].append(unit_subpart)
                a_subpart_is_opened = False
            unit_content["parts"].append(unit_part)

        # Start a new part
        a_part_is_opened = True
        unit_part = {}

        # Set up the title
        unit_part["titre"] = " ".join(tag.stripped_strings)
        # Intialize arrays
        unit_part["texte"] = []
        unit_part["subparts"] = []

    elif tag.name == 'h3':
        # Traitement d'un titre de sous-partie:

        # Stop the current subpart ?
        if a_subpart_is_opened:
            unit_part["subparts"].append(unit_subpart)

        # Start a new subpart
        a_subpart_is_opened = True
        unit_subpart = {}

        # Set up the title
        unit_subpart["titre"] = " ".join(tag.stripped_strings)
        # Initialize arrays
        unit_subpart["texte"] = []

    else:
        # D'autres types de balises sont pour le moment ignorées. Elles pourront être traitées ultérieurement
        print(f"Balise ignorée: {tag.name}")

else:
    # Finalisation de l'inspection et du dictionnaire créé

    # Stop the current part ?
    if a_part_is_opened:
        # Stop the current subpart ?
        if a_subpart_is_opened:
            unit_part["subparts"].append(unit_subpart)
            a_subpart_is_opened = False
        unit_content["parts"].append(unit_part)

unit_content


p
h2
p
div
Balise ignorée: div
div
Balise ignorée: div
h2
p
h3
p
p
h3
p
p
h3
p
p
h2
p
ul
Balise ignorée: ul


{'titre': 'Qu’est-ce que l’ingénierie des données ?',
 'duree': '5 minutes',
 'texte': ['L’ingénieur données travaillera souvent avec plusieurs types de données pour effectuer de nombreuses opérations à l’aide de nombreux langages de script ou de codage appropriés à leur organisation individuelle.'],
 'parts': [{'titre': 'Types de données',
   'texte': ["Il existe trois types principaux de données qu'un ingénieur de données utilisera."],
   'subparts': []},
  {'titre': 'Opérations de données',
   'texte': ['En tant qu’ingénieur données, certaines des tâches principales que vous effectuerez dans Azure incluent l’intégration des données , la transformation des données et la consolidation des données .'],
   'subparts': [{'titre': 'Intégration de données',
     'texte': ['L’intégration des données implique l’établissement de liens entre les services opérationnels et analytiques et les sources de données pour permettre un accès sécurisé et fiable aux données sur plusieurs systèmes. Par exe

## **pymongo**: mise en base de données **mongoDB**

Maintenant que nous avons réussi à extraire un certain nombre d'informations, il est temps de chercher à les mettre en base de données. Pour cela nous utilisons la techno **mongoDB** pour une base de données NoSQL. Avant de procéder à la mise en base de données à proprement parler, il est nécessaire de penser à la structure de notre base de données.

Voici le schéma réalisé pour ce projet

<img src="mongoDB_schema-min.png" alt="schema mongoDB">

*Note: il manque **titre** comme attribut aux collections **parcours_apprentissage** et **module_cours**, ainsi que **durée** et **metadata** à la collection **notion_cours**.*

L'idée est d'avoir trois collections: parcours d'apprentissage, module de cours et notions de cours. Une notion de cours peut être une unité de cours, mais aussi une partie d'unité de cours ou encore une sous-partie. Une notion de cours est ainsi elle-même composée de notions de cours.

### Connection à la base de données

Commençons d'abord par nous connecter à la base de données:

In [21]:
client = MongoClient("mongodb://localhost:27017/")
db = client["webscrapping_microsoft_dp203t00"]

### Création de la collection **parcours_apprentissage** et mise en base du premier document

Créons maintenant un schéma pour une première collection de notre base de données. Commençons par exemple par un parcours d'apprentissage.

In [22]:
schema_parcours = {
    "bsonType": "object",
    "required": [
        "titre",
        "duree",
        "avant_propos",
        "niveau",
        "mots_cles",
    ],
    "properties": {
        "titre": {
            "bsonType": "string",
        },
        "duree": {
            "bsonType": "string",
        },
        "avant_propos": {
            "bsonType": "string",
        },
        "niveau": {
            "bsonType": "string",
        },
        "mots_cles": {
            "bsonType": "array",
            "minItems": 1,
            "items": {
                "bsonType": "string"
            },
        },
        "prerequis": {
            "bsonType": "string",
        },
        "module_ids": {
            "bsonType": "array",
            "minItems": 0,
            "items": {
                "bsonType": "objectId",
            },
        }
    }
}

validator = {"$jsonSchema": schema_parcours}

On va maintenant créer la collection dans la base de données.

*Note: Afin que la cellule suivante fonctionne plusieurs fois d'affilée, si la collection existe déjà, on la supprime pour le récréer.*

In [23]:
try:
    collection = db.create_collection("parcours_apprentissage", validator=validator)
except:
    db.parcours_apprentissage.drop()
    collection = db.create_collection("parcours_apprentissage", validator=validator)

On peut maintenant accéder à la collection et insérer un nouveau parcours

In [24]:
collection = db["parcours_apprentissage"]
# Note: le dictionnaire new_learning_path avait été créé préalablement
learning_path_result = collection.insert_one(new_learning_path)

### Création de la collection **module_cours** et mise en base de son premier document

Ensuite on peut faire de même pour la création d'un premier module dans la base de données. D'abord le schéma.

In [25]:
schema_module = {
    "bsonType": "object",
    "required": [
        "titre",
        "duree",
        "avant_propos",
        "niveau",
        "objectifs",
        "parcours_ids",
        "notion_ids",
    ],
    "properties": {
        "titre": {
            "bsonType": "string",
        },
        "duree": {
            "bsonType": "string",
        },
        "avant_propos": {
            "bsonType": "string",
        },
        "niveau": {
            "bsonType": "string",
        },
        "mots_cles": {
            "bsonType": "array",
            "minItems": 1,
            "items": {
                "bsonType": "string"
            },
        },
        "objectifs": {
            "bsonType": "array",
            "minItems": 1,
            "items": {
                "bsonType": "string"
            },
        },
        "prerequis": {
            "bsonType": "string",
        },
        "parcours_ids": {
            "bsonType": "array",
            "minItems": 0,
            "items": {
                "bsonType": "objectId",
            },
        },
        "notion_ids": {
            "bsonType": "array",
            "minItems": 0,
            "items": {
                "bsonType": "objectId",
            },
        }
    }
}

validator = {"$jsonSchema": schema_module}

Créons la collection

In [26]:
try:
    collection = db.create_collection("module_cours", validator=validator)
except:
    db.module_cours.drop()
    collection = db.create_collection("module_cours", validator=validator)

et insérons un premier module préalablement aggrégé dans un dictionnaire. On va au préalable ajouter l'ID du parcours

In [27]:
# Note: le dictionnaire new_module avait été créé préalablement
new_module["parcours_ids"].append(learning_path_result.inserted_id)

collection = db["module_cours"]
module_result = collection.insert_one(new_module)

Maintenant que le module est créé, nous pouvons également insérer son ID dans le document **parcours_apprentissage** créé précédemment.

In [28]:
my_fields = {
    "$push": {"module_ids": module_result.inserted_id},
}
my_filter = {
    "_id": learning_path_result.inserted_id,
}
collection = db["parcours_apprentissage"]
collection.update_one(my_filter, my_fields)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### Création de la collection **notion_cours** et mise en base de ses premiers documents

et pour terminer, nous allons mettre en base de données le contenu d'une unité de cours de ce module

In [29]:
schema_notion = {
    "bsonType": "object",
    "required": [
        "titre",
        "texte"
    ],
    "properties": {
        "titre": {
            "bsonType": "string",
        },
        "duree": {
            "bsonType": "string",
        },
        "metadata": {
            "bsonType": "array",
            "minItems": 0,
            "items": {
                "bsonType": "string",
            },
        },
        "texte": {
            "bsonType": "array",
            "minItems": 1,
            "items": {
                "bsonType": "string",
            },
        },
        "notion_ids": {
            "bsonType": "array",
            "minItems": 0,
            "items": {
                "bsonType": "objectId",
            },
        }
    }
}

validator = {"$jsonSchema": schema_notion}

Créons la collection

In [30]:
try:
    collection = db.create_collection("notion_cours", validator=validator)
except:
    db.notion_cours.drop()
    collection = db.create_collection("notion_cours", validator=validator)

L'insertion du contenu d'une page est un peu plus complexe que précédemment car ils vont être mises en base en plusieurs documents. Pour rappel le contenu d'une unité de cours du module déjà en base est insérée dans:

In [31]:
unit_content

{'titre': 'Qu’est-ce que l’ingénierie des données ?',
 'duree': '5 minutes',
 'texte': ['L’ingénieur données travaillera souvent avec plusieurs types de données pour effectuer de nombreuses opérations à l’aide de nombreux langages de script ou de codage appropriés à leur organisation individuelle.'],
 'parts': [{'titre': 'Types de données',
   'texte': ["Il existe trois types principaux de données qu'un ingénieur de données utilisera."],
   'subparts': []},
  {'titre': 'Opérations de données',
   'texte': ['En tant qu’ingénieur données, certaines des tâches principales que vous effectuerez dans Azure incluent l’intégration des données , la transformation des données et la consolidation des données .'],
   'subparts': [{'titre': 'Intégration de données',
     'texte': ['L’intégration des données implique l’établissement de liens entre les services opérationnels et analytiques et les sources de données pour permettre un accès sécurisé et fiable aux données sur plusieurs systèmes. Par exe

Le contenu étant déjà sous forme de dictionnaires de dictionnaires, nous allons pouvoir faire la mise en base de tous les documents "notions_cours" contenus dans ce dictionnaire et les relier les uns aux autres.

In [32]:
collection = db["notion_cours"]

parts = []
for part in unit_content["parts"]:
    subparts = []
    for subpart in part["subparts"]:
        notion_result = collection.insert_one(subpart)
        subparts.append(notion_result.inserted_id)

    if len(subparts) > 0:
        # On remplace les données déjà mises en base par leur ID nouvellement créé
        part["subparts"] = subparts
    else:
        part.pop("subparts")

    notion_result = collection.insert_one(part)
    parts.append(notion_result.inserted_id)

if len(parts) > 0:
    # On remplace les données déjà mises en base par leur ID nouvellement créé
    unit_content["parts"] = parts
else:
    unit_content.pop("parts")

notion_result = collection.insert_one(unit_content)      

Maintenant que l'unité de cours a été mise en base, on peut indiquer son Id dans le document du module correspondant.

In [33]:
my_fields = {
    "$push": {"notion_ids": notion_result.inserted_id},
}
my_filter = {
    "_id": module_result.inserted_id,
}
collection = db["module_cours"]
collection.update_one(my_filter, my_fields)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

## Pour poursuivre le travail...

Ce notebook montre les différentes étapes à suivre pour appliquer la méthode de webscrapping à partir du site https://learn.microsoft.com/fr-fr/training/courses/dp-203t00
L'exemple des commandes ci-dessus s'applique à l'extraction des informations d'une page de parcours référencée sur cette page, puis d'une page module référencée sur la page parcours, et enfin d'une page unité de cours référencée sur la page du module.

Pour aller plus loin, il serait nécessaire de mettre ce code sous forme de script pour pouvoir l'appliquer à plus de pages. Cela permettrait en outre de tester si la méthode développée ne rencontre pas de problèmes sur d'autres pages qu'on a supposé être structuré de la même manière que les pages utilisées dans ce notebook.

Ensuite, l'ensemble du contenu pertinent des pages n'a pas été entièrement récupéré. Certaines balises HTML ont été ignoré:
- Les listes de texte
- Les tableaux
- Les images
sont par exemple des balises qu'il faudrait essayer d'extraire.

Nous avons également supposé deux niveaux de parties et sous-parties pour une unité de cours. Peut-être certaines unités possèdent une couche supplémentaire.

Enfin certaines unités de cours ne sont pas pertinentes à extraire (par exemple *Évaluation du module*). Le script à developper devrait prendre soin de ne pas extraire celle-là.